In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [2]:
df=pd.read_csv('CKD.csv')
df=pd.get_dummies(df,dtype=int,drop_first=True)
df

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [3]:
df.isnull().values.any()

np.False_

In [4]:
indep=df[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
dep=df['classification_yes']

In [5]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(indep)

SUPPORT_VECTOR_MACHINE

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
param_grid={'kernel':['rbf','sigmoid','linear'],
              'C':[1,10],'gamma':['auto','scale']}
grid=GridSearchCV(SVC(probability=True),param_grid,refit = True, verbose = 3, n_jobs=-1, scoring='f1_weighted')
grid.fit(indep,dep)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Users\Hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\ma\core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [1, 10], 'gamma': ['auto', 'scale'],
                         'kernel': ['rbf', 'sigmoid', 'linear']},
             scoring='f1_weighted', verbose=3)

In [9]:
re=grid.cv_results_
y_pred=grid.predict(indep)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(dep,y_pred)
print(cm)

from sklearn.metrics import classification_report
cr=classification_report(dep,y_pred)
print(cr)

from sklearn.metrics import f1_score
fs=f1_score(dep,y_pred,average='weighted')
print("The f1_macro value for the best parameters {}:".format(grid.best_params_),fs)

from sklearn.metrics import roc_auc_score
roc_score=roc_auc_score(dep,grid.predict_proba(indep)[:,1])
print("The roc_score:",roc_score)


[[144   6]
 [  3 246]]
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       150
           1       0.98      0.99      0.98       249

    accuracy                           0.98       399
   macro avg       0.98      0.97      0.98       399
weighted avg       0.98      0.98      0.98       399

The f1_macro value for the best parameters {'C': 10, 'gamma': 'auto', 'kernel': 'linear'}: 0.9773972219607265
The roc_score: 0.9976171352074965


In [10]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.094411,0.003052,0.013984,0.002471,1,auto,rbf,"{'C': 1, 'gamma': 'auto', 'kernel': 'rbf'}",0.480769,0.480769,0.480769,0.480769,0.474881,0.479592,0.002355,8
1,0.032832,0.005016,0.008419,0.002192,1,auto,sigmoid,"{'C': 1, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.480769,0.480769,0.480769,0.480769,0.474881,0.479592,0.002355,8
2,119.863741,66.457270,0.005374,0.000872,1,auto,linear,"{'C': 1, 'gamma': 'auto', 'kernel': 'linear'}",0.889143,0.987539,0.962059,0.987456,1.000000,0.965240,0.040000,3
3,0.056706,0.002300,0.011694,0.002000,1,scale,rbf,"{'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}",0.480769,0.480769,0.480769,0.480769,0.474881,0.479592,0.002355,8
4,0.036604,0.001462,0.005729,0.000505,1,scale,sigmoid,"{'C': 1, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.500141,0.496055,0.410714,0.515785,0.566398,0.497819,0.050243,7
5,119.175272,53.218970,0.004690,0.000397,1,scale,linear,"{'C': 1, 'gamma': 'scale', 'kernel': 'linear'}",0.889143,0.987539,0.962059,0.987456,1.000000,0.965240,0.040000,3
6,0.098015,0.003179,0.013188,0.000882,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",0.480769,0.480769,0.480769,0.480769,0.474881,0.479592,0.002355,8
7,0.029513,0.006161,0.004895,0.000374,10,auto,sigmoid,"{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.480769,0.480769,0.480769,0.480769,0.474881,0.479592,0.002355,8
8,150.972756,49.875282,0.004383,0.001229,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",0.901354,1.000000,0.962059,0.974814,1.000000,0.967645,0.036251,1
9,0.056244,0.005308,0.009611,0.000526,10,scale,rbf,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",0.528290,0.480769,0.544794,0.480769,0.474881,0.501901,0.028842,6
